<h1>Практическая работа №3<h1>
<h2>Выполнила: Бардасова Ирина<h2>
<h2>Группа: ББМО-02-22<h2/>

 **Шаг 1**
Скопировать проект по ссылке в локальную среду выполнения Jupyter (Google Colab).

In [2]:
!git clone https://github.com/ewatson2/EEL6812_DeepFool_Project

fatal: destination path 'EEL6812_DeepFool_Project' already exists and is not an empty directory.


**Шаг 2**
Сменить директорию исполнения на вновь созданную папку "EEL6812_DeepFool_Project" проекта.

In [3]:
%cd EEL6812_DeepFool_Project/

/content/EEL6812_DeepFool_Project


**Шаг 3**
Выполнить импорт библиотек.

In [4]:
!pip install numpy
!pip install art
!pip install adversarial-robustness-toolbox

In [5]:
import numpy as np

import tensorflow as tf

from art.attacks.poisoning.backdoor_attack_dgm.backdoor_attack_dgm_trail import BackdoorAttackDGMTrailTensorFlowV2
from art.estimators.gan.tensorflow import TensorFlowV2GAN
from art.estimators.generation.tensorflow import TensorFlowV2Generator
from art.estimators.classification.tensorflow import TensorFlowV2Classifier

np.random.seed(100)
tf.random.set_seed(100)

**Шаг 4**
Создадим класс для модели-генератора изображений.

In [6]:
def make_generator_model(capacity: int, z_dim: int) -> tf.keras.Sequential():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Dense(\
                                  capacity * 7 * 7 * 4,\
                                  use_bias=False,\
                                  input_shape=(z_dim,)))

  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Reshape((7, 7, capacity * 4)))
  assert model.output_shape == (None, 7, 7, capacity * 4)

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            capacity * 2,\
                                            (5, 5),\
                                            strides=(1, 1),\
                                            padding="same",\
                                            use_bias=False))

  assert model.output_shape == (None, 7, 7, capacity * 2)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            capacity,\
                                            (5, 5),\
                                            strides=(2, 2),\
                                            padding="same",\
                                            use_bias=False))

  assert model.output_shape == (None, 14, 14, capacity)
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Conv2DTranspose(\
                                            1,\
                                            (5, 5),\
                                            strides=(2, 2),\
                                            padding="same",\
                                            use_bias=False))

  model.add(tf.keras.layers.Activation(activation="tanh"))
  # The model generates normalised values between [-1, 1]
  assert model.output_shape == (None, 28, 28, 1)

  return model

**Шаг 5**
Создадим класс для модели-дискриминатора изображений.

In [7]:
def make_discriminator_model(capacity: int) -> tf.keras.Sequential():
  model = tf.keras.Sequential()

  model.add(tf.keras.layers.Conv2D(\
                                   capacity,\
                                   (5, 5),\
                                   strides=(2, 2),\
                                   padding="same",\
                                   input_shape=[28, 28, 1]))

  model.add(tf.keras.layers.LeakyReLU())

  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Conv2D(\
                                   capacity * 2,\
                                   (5, 5),\
                                   strides=(2, 2),\
                                   padding="same"))

  model.add(tf.keras.layers.LeakyReLU())
  model.add(tf.keras.layers.Dropout(0.3))

  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.Dense(1))

  return model

**Шаг 6**
Создадим атакующий триггер.

In [8]:
z_trigger = np.random.randn(1, 94).astype(np.float64) # 59 + 35 (мой порядковый номер в списке)

**Шаг 7**
Загрузим датасет MNIST.

In [9]:
# load dataset
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype("float32")

# Normalize the images in between -1 and 1
train_images = (train_images - 127.5) / 127.5

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

**Шаг 8**
Создаём цель атаки.

In [10]:
x_target = train_images[35:36,:][0]

**Шаг 9**
Определим функцию потерь дискриминатора.

In [11]:
# Define discriminator loss
def discriminator_loss(true_output, fake_output):
  true_loss = cross_entropy(tf.ones_like(true_output), true_output)
  fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
  tot_loss = true_loss + fake_loss

  return tot_loss

**Шаг 10**
Определим функцию потерь генератора.

In [12]:
# Define Generator loss
def generator_loss(fake_output):
  return cross_entropy(tf.ones_like(fake_output), fake_output)

noise_dim = 94
capacity = 64
generator = TensorFlowV2Generator(\
                            encoding_length=noise_dim, \
                            model=make_generator_model(capacity, noise_dim))
discriminator_classifier = TensorFlowV2Classifier(\
                                  model=make_discriminator_model(capacity),\
                                  nb_classes=2, \
                                  input_shape=(28, 28, 1))

**Шаг 11**
Создаём генератор.

In [13]:
# Build GAN
gan = TensorFlowV2GAN(\
                generator=generator,\
                discriminator=discriminator_classifier, \
                generator_loss=generator_loss,\
                generator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),\
                discriminator_loss=discriminator_loss,\
                discriminator_optimizer_fct=tf.keras.optimizers.Adam(1e-4),\
                )

**Шаг 12**
Создаём атаку на генератор.

In [16]:
# Create BackDoorAttacks Class
gan_attack = BackdoorAttackDGMTrailTensorFlowV2(gan=gan)
print("Poisoning estimator")
poisoned_generator = gan_attack.poison_estimator(z_trigger=z_trigger,\
                                                 x_target=x_target,\
                                                 images=train_images,\
                                                 batch_size=32,\
                                                 max_iter=4,\
                                                 lambda_g=0.1,\
                                                 verbose=2)
print("Finished poisoning estimator")

Poisoning estimator


Finished poisoning estimator


**Шаг 13**
Выполним оценку точности атаки.

In [17]:
# Check the success rate
x_pred_trigger = poisoned_generator.model(z_trigger)[0]
print("Target Fidelity (Attack Objective): %.2f%%" %
np.sum((x_pred_trigger - x_target) ** 2))

Target Fidelity (Attack Objective): 12.19%


**Шаг 14**
Сохраним артефакты атаки.

In [18]:
# Save trigger, target and save the model
np.save("z_trigger_trail.npy", z_trigger)
np.save("x_target_trail.npy", x_target)
poisoned_generator.model.save("trail-mnist-dcgan")

***Заключение***: Метод проведения атаки на GAN представляет из себя мощный инструмент для внедрения вредоносного поведения в системе.
*Что происходит:* Генератор обучается генерировать изображения, которые похожи на изображения из обучающего набора данных. В процессе "отравления" генератора, он модифицируется таким образом, чтобы при подаче на вход определенного триггера z_trigger, он генерировал специфическое целевое изображение x_target.
Если триггер будет выбран таким образом, чтобы он не был очевидным в сгенерированных изображениях, атака может оставаться незамеченной.